Code to get and clean data from Mt. Washington weather station

In [47]:
#Import required modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [48]:
#Load in raw Mt. Washington station observations
mwobs = pd.read_csv('C:/users/benja/downloads/raw_station_data_UTCtime.csv')

C:\Users\benja\AppData\Local\Temp\ipykernel_30136\1479476298.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  mwobs = pd.read_csv('C:/users/benja/downloads/raw_station_data_UTCtime.csv')


In [49]:
#Convert observation time to datetime
mwobs['valid'] = pd.to_datetime(mwobs['valid'])

In [50]:
#List columns of dataset
mwobs.columns

Index(['station', 'valid', 'tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'p01i',
       'alti', 'mslp', 'vsby', 'gust', 'skyc1', 'skyc2', 'skyc3', 'skyc4',
       'skyl1', 'skyl2', 'skyl3', 'skyl4', 'wxcodes', 'ice_accretion_1hr',
       'ice_accretion_3hr', 'ice_accretion_6hr', 'peak_wind_gust',
       'peak_wind_drct', 'peak_wind_time', 'feel', 'metar', 'snowdepth'],
      dtype='object')

In [51]:
#Filter to select only times within 30mins of hours of interest
condition = ((mwobs['valid'].dt.hour == 4) & (mwobs['valid'].dt.minute <= 30)) | \
            ((mwobs['valid'].dt.hour == 3) & (mwobs['valid'].dt.minute >= 30)) | \
            ((mwobs['valid'].dt.hour == 10) & (mwobs['valid'].dt.minute <= 30)) | \
            ((mwobs['valid'].dt.hour == 9) & (mwobs['valid'].dt.minute >= 30)) | \
            ((mwobs['valid'].dt.hour == 19) & (mwobs['valid'].dt.minute >= 30)) | \
            ((mwobs['valid'].dt.hour == 20) & (mwobs['valid'].dt.minute <= 30))

filtered_mwobs = mwobs[condition]

In [52]:
filtered_mwobs.reset_index(inplace=True, drop=True)


In [53]:
#Drop columns that we determined were likely not particularly relevant, contained many missing values, were repetitive, etc.
mwobs_init = filtered_mwobs.drop(columns = ['station', 'metar', 'skyc1', 'skyc2', 'skyc3', 'skyc4',
       'skyl1', 'skyl2', 'skyl3', 'skyl4', 'ice_accretion_1hr',
       'ice_accretion_3hr', 'peak_wind_time', 'feel', 'metar', 'snowdepth', 'peak_wind_gust', 'peak_wind_drct', 'alti', 'mslp', 'ice_accretion_6hr', 'p01i'])

In [54]:
#Drop rows where temperature data is missing
mwobs_init = mwobs_init.drop(mwobs_init['tmpf'][mwobs_init['tmpf'] == 'M'].index)

In [55]:
mwobs_init

,valid,tmpf,dwpf,relh,drct,sknt,vsby,gust,wxcodes
0,2014-08-01 03:51:00,44.60,44.60,100.00,270.00,27.00,0.00,M,FG
1,2014-08-01 09:56:00,44.60,42.80,93.35,280.00,23.00,0.50,M,M
2,2014-08-01 19:57:00,55.40,46.40,71.64,230.00,13.00,6.00,M,HZ
3,2014-08-02 03:57:00,50.00,46.40,87.37,240.00,26.00,15.00,M,M
4,2014-08-02 09:53:00,51.80,44.60,76.34,210.00,8.00,20.00,M,M
...,...,...,...,...,...,...,...,...,...
10480,2024-04-24 09:50:00,28.40,28.40,100.00,270.00,44.0,0.00,50.00,-SN FZFG DRSN
10481,2024-04-24 19:50:00,19.40,19.40,100.00,290.00,43.0,0.00,49.00,-SHGS FZFG BLSN
10482,2024-04-25 03:47:00,1.40,-2.20,84.43,320.00,55.0,0.12,63.00,FZFG BLSN
10483,2024-04-25 09:54:00,6.80,-2.20,65.87,330.00,51.0,80.00,57.00,M


In [56]:
#If wind gust data is missing, replace missing values with wind speed from same time
mwobs_init.loc[mwobs_init['gust'] == 'M', 'gust'] = mwobs_init['sknt']

In [57]:
#List unique values for wxcodes (present weather field) and frequency of occurrence in dataset
mwobs_init['wxcodes'].value_counts()[0:15]

wxcodes
M                  4132
FG                 1609
FZFG               1124
FZFG BLSN           483
-SN FZFG BLSN       364
-SHRA FG            291
-SHSN FZFG BLSN     243
-RA FG              228
BCFG                207
-SN FZFG DRSN       168
-SN FZFG            163
-SHSN FZFG          143
-DZ FG              106
FZFG DRSN            83
SN FZFG BLSN         74
Name: count, dtype: int64

In [58]:
#One-hot encode a subset of wxcodes. Include one-hot encoding for missing wxcode since it likely means no signifcant weather
#Missing values here are informative
mwobs_init.insert(1, 'SN', mwobs_init['wxcodes'].str.contains('SN'))
mwobs_init.insert(1, 'RA', mwobs_init['wxcodes'].str.contains('RA'))
mwobs_init.insert(1, 'FG', mwobs_init['wxcodes'].str.contains('FG'))
mwobs_init.insert(1, 'BL', mwobs_init['wxcodes'].str.contains('BL'))
mwobs_init.insert(1, 'M', mwobs_init['wxcodes'].str.contains('M'))


In [59]:
mwobs_init = mwobs_init.drop(columns = 'wxcodes')

In [60]:
mwobs_init

,valid,M,BL,FG,RA,SN,tmpf,dwpf,relh,drct,sknt,vsby,gust
0,2014-08-01 03:51:00,False,False,True,False,False,44.60,44.60,100.00,270.00,27.00,0.00,27.00
1,2014-08-01 09:56:00,True,False,False,False,False,44.60,42.80,93.35,280.00,23.00,0.50,23.00
2,2014-08-01 19:57:00,False,False,False,False,False,55.40,46.40,71.64,230.00,13.00,6.00,13.00
3,2014-08-02 03:57:00,True,False,False,False,False,50.00,46.40,87.37,240.00,26.00,15.00,26.00
4,2014-08-02 09:53:00,True,False,False,False,False,51.80,44.60,76.34,210.00,8.00,20.00,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480,2024-04-24 09:50:00,False,False,True,False,True,28.40,28.40,100.00,270.00,44.0,0.00,50.00
10481,2024-04-24 19:50:00,False,True,True,False,True,19.40,19.40,100.00,290.00,43.0,0.00,49.00
10482,2024-04-25 03:47:00,False,True,True,False,True,1.40,-2.20,84.43,320.00,55.0,0.12,63.00
10483,2024-04-25 09:54:00,True,False,False,False,False,6.80,-2.20,65.87,330.00,51.0,80.00,57.00


In [61]:
#Remove rows in which wind direction is missing
mwobs_init.drop((mwobs_init[mwobs_init['drct'] == 'M']).index, inplace=True)

In [62]:
mwobs_init.reset_index(inplace=True)

In [63]:
mwobs_init

,index,valid,M,BL,FG,RA,SN,tmpf,dwpf,relh,drct,sknt,vsby,gust
0,0,2014-08-01 03:51:00,False,False,True,False,False,44.60,44.60,100.00,270.00,27.00,0.00,27.00
1,1,2014-08-01 09:56:00,True,False,False,False,False,44.60,42.80,93.35,280.00,23.00,0.50,23.00
2,2,2014-08-01 19:57:00,False,False,False,False,False,55.40,46.40,71.64,230.00,13.00,6.00,13.00
3,3,2014-08-02 03:57:00,True,False,False,False,False,50.00,46.40,87.37,240.00,26.00,15.00,26.00
4,4,2014-08-02 09:53:00,True,False,False,False,False,51.80,44.60,76.34,210.00,8.00,20.00,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10449,10480,2024-04-24 09:50:00,False,False,True,False,True,28.40,28.40,100.00,270.00,44.0,0.00,50.00
10450,10481,2024-04-24 19:50:00,False,True,True,False,True,19.40,19.40,100.00,290.00,43.0,0.00,49.00
10451,10482,2024-04-25 03:47:00,False,True,True,False,True,1.40,-2.20,84.43,320.00,55.0,0.12,63.00
10452,10483,2024-04-25 09:54:00,True,False,False,False,False,6.80,-2.20,65.87,330.00,51.0,80.00,57.00


In [64]:
#Iterate through wind directions, turn wind direction in degrees into direction

wdirs = mwobs_init['drct'][:]
wdirs_int = []
#Turn wind directions into integers
for i in range(len(wdirs)):
  wdirs_int.append(int(eval(wdirs[i])))

wdirs = np.array(wdirs_int)

#Set up centers of 45-degree octants of wind direction
wdir_vec = [0, 45, 90, 135, 180, 225, 270, 315, 360]
wdir_cats = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N']

#Find which center value the wind direction is closest to and store corresponding string containing wind direction
new_wd = []
for i in range(len(wdirs)):
  wdir_obs_vec = [wdirs[i] for j in range(len(wdir_vec))]
  idxmin = np.argmin(np.abs(np.array(wdir_obs_vec) - np.array(wdir_vec)))
  new_wd.append(wdir_cats[idxmin])

mwobs_init.insert(1, 'WindDir', new_wd)
mwobs_init.drop(columns = 'drct', inplace=True)

In [65]:
mwobs_init.drop(columns = 'index', inplace=True)
mwobs_init

,WindDir,valid,M,BL,FG,RA,SN,tmpf,dwpf,relh,sknt,vsby,gust
0,W,2014-08-01 03:51:00,False,False,True,False,False,44.60,44.60,100.00,27.00,0.00,27.00
1,W,2014-08-01 09:56:00,True,False,False,False,False,44.60,42.80,93.35,23.00,0.50,23.00
2,SW,2014-08-01 19:57:00,False,False,False,False,False,55.40,46.40,71.64,13.00,6.00,13.00
3,SW,2014-08-02 03:57:00,True,False,False,False,False,50.00,46.40,87.37,26.00,15.00,26.00
4,SW,2014-08-02 09:53:00,True,False,False,False,False,51.80,44.60,76.34,8.00,20.00,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10449,W,2024-04-24 09:50:00,False,False,True,False,True,28.40,28.40,100.00,44.0,0.00,50.00
10450,W,2024-04-24 19:50:00,False,True,True,False,True,19.40,19.40,100.00,43.0,0.00,49.00
10451,NW,2024-04-25 03:47:00,False,True,True,False,True,1.40,-2.20,84.43,55.0,0.12,63.00
10452,NW,2024-04-25 09:54:00,True,False,False,False,False,6.80,-2.20,65.87,51.0,80.00,57.00


In [66]:
#Remove rows with missing relative humidity values
mwobs_init.drop(mwobs_init[mwobs_init['relh'] == 'M'].index, inplace=True)
mwobs_init

,WindDir,valid,M,BL,FG,RA,SN,tmpf,dwpf,relh,sknt,vsby,gust
0,W,2014-08-01 03:51:00,False,False,True,False,False,44.60,44.60,100.00,27.00,0.00,27.00
1,W,2014-08-01 09:56:00,True,False,False,False,False,44.60,42.80,93.35,23.00,0.50,23.00
2,SW,2014-08-01 19:57:00,False,False,False,False,False,55.40,46.40,71.64,13.00,6.00,13.00
3,SW,2014-08-02 03:57:00,True,False,False,False,False,50.00,46.40,87.37,26.00,15.00,26.00
4,SW,2014-08-02 09:53:00,True,False,False,False,False,51.80,44.60,76.34,8.00,20.00,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10449,W,2024-04-24 09:50:00,False,False,True,False,True,28.40,28.40,100.00,44.0,0.00,50.00
10450,W,2024-04-24 19:50:00,False,True,True,False,True,19.40,19.40,100.00,43.0,0.00,49.00
10451,NW,2024-04-25 03:47:00,False,True,True,False,True,1.40,-2.20,84.43,55.0,0.12,63.00
10452,NW,2024-04-25 09:54:00,True,False,False,False,False,6.80,-2.20,65.87,51.0,80.00,57.00


In [67]:
#Removerows with missing visibility values (only 1 row)
mwobs_init.drop(mwobs_init[mwobs_init['vsby'] == 'M'].index, inplace=True)
mwobs_init

,WindDir,valid,M,BL,FG,RA,SN,tmpf,dwpf,relh,sknt,vsby,gust
0,W,2014-08-01 03:51:00,False,False,True,False,False,44.60,44.60,100.00,27.00,0.00,27.00
1,W,2014-08-01 09:56:00,True,False,False,False,False,44.60,42.80,93.35,23.00,0.50,23.00
2,SW,2014-08-01 19:57:00,False,False,False,False,False,55.40,46.40,71.64,13.00,6.00,13.00
3,SW,2014-08-02 03:57:00,True,False,False,False,False,50.00,46.40,87.37,26.00,15.00,26.00
4,SW,2014-08-02 09:53:00,True,False,False,False,False,51.80,44.60,76.34,8.00,20.00,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10449,W,2024-04-24 09:50:00,False,False,True,False,True,28.40,28.40,100.00,44.0,0.00,50.00
10450,W,2024-04-24 19:50:00,False,True,True,False,True,19.40,19.40,100.00,43.0,0.00,49.00
10451,NW,2024-04-25 03:47:00,False,True,True,False,True,1.40,-2.20,84.43,55.0,0.12,63.00
10452,NW,2024-04-25 09:54:00,True,False,False,False,False,6.80,-2.20,65.87,51.0,80.00,57.00


In [68]:
mwobs_init.reset_index(drop = True, inplace=True)

In [69]:
mwobs_data = mwobs_init

In [70]:
mwobs_data['valid'] = pd.to_datetime(mwobs_data['valid'])

In [71]:
#Split into 11pm (4UTC), 5am (10UTC), and 3pm (20UTC) data
mwobs_11pm = mwobs_data[mwobs_data['valid'].dt.hour < 6]
mwobs_5am_part1 = mwobs_data[mwobs_data['valid'].dt.hour < 13]
mwobs_5am = mwobs_5am_part1[mwobs_5am_part1['valid'].dt.hour > 7]
mwobs_3pm = mwobs_data[mwobs_data['valid'].dt.hour > 14]

In [72]:
mwobs_3pm.reset_index(inplace=True, drop=True)
mwobs_5am.reset_index(inplace=True, drop=True)
mwobs_11pm.reset_index(inplace=True, drop=True)


In [73]:
#Iterate through dates within date range
#If data is available for 4UTC, 10UTC, and 20UTC on the same day, append data point to new dataframe
daterange = pd.date_range('2014-08-02', '2024-04-10')
mwobs_5am_shared = pd.DataFrame(mwobs_5am.iloc[0]).transpose()
mwobs_3pm_shared = pd.DataFrame(mwobs_3pm.iloc[0]).transpose()
mwobs_11pm_shared = pd.DataFrame(mwobs_11pm.iloc[0]).transpose()
for i in range(0, len(daterange)):
  if mwobs_5am[mwobs_5am['valid'].dt.date == daterange[i].date()].shape[0] > 0:
    if mwobs_3pm[mwobs_3pm['valid'].dt.date == daterange[i].date()].shape[0] > 0:
      if mwobs_11pm[mwobs_11pm['valid'].dt.date == daterange[i].date()].shape[0] > 0:
        mwobs_5am_shared = pd.concat([mwobs_5am_shared, mwobs_5am[mwobs_5am['valid'].dt.date == daterange[i].date()]], axis = 0)
        mwobs_3pm_shared = pd.concat([mwobs_3pm_shared, mwobs_3pm[mwobs_3pm['valid'].dt.date == daterange[i].date()]], axis = 0)
        mwobs_11pm_shared = pd.concat([mwobs_11pm_shared, mwobs_11pm[mwobs_11pm['valid'].dt.date == daterange[i].date()]], axis = 0)

In [74]:
mwobs_5am_shared.reset_index(inplace=True, drop=True)
mwobs_3pm_shared.reset_index(inplace=True, drop=True)
mwobs_11pm_shared.reset_index(inplace=True, drop=True)

In [75]:
#Rename wind direction column for all dataframes
wd_temp_5am = mwobs_5am_shared['WindDir']
mwobs_5am_shared.drop(columns = ['WindDir'], inplace=True)
mwobs_5am_shared.insert(12, 'wd', wd_temp_5am)

wd_temp_3pm = mwobs_3pm_shared['WindDir']
mwobs_3pm_shared.drop(columns = ['WindDir'], inplace=True)
mwobs_3pm_shared.insert(12, 'wd', wd_temp_3pm)

wd_temp_11pm = mwobs_11pm_shared['WindDir']
mwobs_11pm_shared.drop(columns = ['WindDir'], inplace=True)
mwobs_11pm_shared.insert(12, 'wd', wd_temp_11pm)


In [76]:
#Check why 11pm (4UTC) dataframe has one more value than the others
#valud_counts() reveals it has two observations from same hour and date
mwobs_11pm_shared['valid'].dt.date.value_counts()

valid
2023-10-18    2
2014-08-01    1
2021-03-03    1
2021-02-08    1
2021-02-09    1
             ..
2017-11-01    1
2017-11-02    1
2017-11-03    1
2017-11-04    1
2024-04-10    1
Name: count, Length: 3340, dtype: int64

In [77]:
#Find the second observation from 4UTC on 10/18/2023
mwobs_11pm_shared.iloc[3167]

valid    2023-10-18 03:57:00
M                      False
BL                     False
FG                      True
RA                     False
SN                     False
tmpf                   30.20
dwpf                   30.20
relh                  100.00
sknt                    12.0
vsby                    0.00
gust                    12.0
wd                         W
Name: 3167, dtype: object

In [78]:
#Drop this observation
mwobs_11pm_shared.drop(index = 3167, inplace=True)

In [79]:
mwobs_11pm_shared.reset_index(inplace=True, drop=True)

In [80]:
#Now the dataset has only one observation per day at 4UTC
mwobs_11pm_shared['valid'].dt.date.value_counts()

valid
2014-08-01    1
2021-04-28    1
2021-02-08    1
2021-02-09    1
2021-02-10    1
             ..
2017-11-01    1
2017-11-02    1
2017-11-03    1
2017-11-04    1
2024-04-10    1
Name: count, Length: 3340, dtype: int64

In [81]:
#Concatenate the 5am (10UTC) and 11pm (4UTC) observations into dataframe, rename columns 
mwobs_features = pd.concat([mwobs_5am_shared, mwobs_11pm_shared], axis = 1)
mwobs_features.columns = ['valid', 'M_5am', 'BL_5am', 'FG_5am', 'RA_5am', 'SN_5am', 't_5am', 'dt_5am', 'rh_5am', 'ws_5am', 'vis_5am', 'gust_5am', 'wd_5am', 'valid_2', 'M_11pm', 'BL_11pm', 'FG_11pm', 'RA_11pm', 'SN_11pm', 't_11pm', 'dt_11pm', 'rh_11pm', 'ws_11pm', 'vis_11pm', 'gust_11pm', 'wd_11pm']

In [82]:
#Drop redundant date column, rename other date column
date = mwobs_features['valid'].dt.date
mwobs_features.drop(columns = ['valid', 'valid_2'], inplace=True)
mwobs_features.insert(0, 'date', date)

In [83]:
#Convert visibilities to floats
mwobs_features['vis_5am'] = mwobs_features['vis_5am'].astype(float)


In [84]:
#Download feature data as a CSV
#mwobs_features.to_csv("C:/users/benja/downloads/ORIE4741/mwobs_feature_data.csv")

In [85]:
#Get label data from time, wind speed, gust data from 3PM (20UTC) observations
mwobs_labels = mwobs_3pm_shared
mwobs_labels = mwobs_labels[['valid', 'sknt', 'gust']]
mwobs_labels.columns = ['date', 'sknt', 'gust']

#Save label data as CSV
mwobs_labels.to_csv('C:/users/benja/downloads/ORIE4741/all_label_data.csv')